In [1]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch 
from torch.utils.data import Dataset
from huggingface_hub import login
from tqdm import tqdm
import  DicesClassificationDataset

In [2]:
# load the data set for few shot prompting
base_dir_name = "RLHF/"
path_to_data = os.getcwd().split(base_dir_name, 1)[0] + base_dir_name + "data/dices/"
train_data_path = path_to_data + "filtered_train_data_white.csv"
validation_path = path_to_data + "filtered_test_data_white.csv"

In [3]:
dataset = DicesClassificationDataset.DicesClassificationPrompts(train_data_path=train_data_path, validation_data_path=validation_path, number_of_prompts=4) 

In [4]:
# to use models such as LLAMA2 and Gemma use login(key) 
# add your own key

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/rk7527/.cache/huggingface/token
Login successful


In [8]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    quantization_config=quantization_config, 
    device_map = device
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.46 GiB (GPU 0; 7.43 GiB total capacity; 5.11 GiB already allocated; 48.25 MiB free; 5.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
responses = []
for i, data in tqdm(enumerate(dataset)):
    templeted_data = tokenizer.apply_chat_template(data[0], return_tensors="pt")
    tokens = templeted_data.to(device)
    outputs = model.generate(tokens, max_new_tokens=30)
    responses.append(tokenizer.decode(outputs[0]))
    print(tokenizer.decode(outputs[0]))
    break # remove the break statment to store the data for 